# Library

In [4]:
# Native library
import copy
import collections
import multiprocessing as mp

import warnings
warnings.filterwarnings('ignore')

import os
import sys
path = os.path.join(os.pardir, os.pardir)
sys.path.append(path)

# Save object
import joblib
import json

from tqdm import tqdm

# Data management
import numpy as np
import pandas as pd
import xarray as xr

import plotly.express as px

from src.constants import TARGET, TARGET_TEST, FOLDER, S_COLUMNS, G_COLUMNS, M_COLUMNS

# Data prepocessing
from src.features.preprocessing import Smoother, Convertor, Scaler
from umap import UMAP
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# Hyperoptimization
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

import wandb

# Regressor models
from xgboost import XGBRegressor

# Model evaluation
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

# Constant

In [5]:
MODEL_PATH = os.path.join('model', FOLDER, 'XGBoost', 'Aggregate')
DATA_PATH = os.path.join(os.pardir, os.pardir, 'data', 'interim', FOLDER)
# os.makedirs(MODEL_PATH, exist_ok=True)

# Import Data

In [6]:
xdf = xr.open_dataset(os.path.join(DATA_PATH, 'train.nc'))
xdf

<xarray.Dataset>
Dimensions:                                         (ts_obs: 557,
                                                     state_dev: 24, ts_aug: 10,
                                                     datetime: 275, name: 3)
Coordinates:
    time                                            (ts_obs, state_dev, ts_aug) object ...
  * ts_obs                                          (ts_obs) int32 0 1 ... 556
  * datetime                                        (datetime) object '2021-1...
  * state_dev                                       (state_dev) int32 23 ... 0
  * ts_aug                                          (ts_aug) int32 0 1 2 ... 8 9
  * name                                            (name) object 'Chau_Phu' ...
Data variables: (12/56)
    red                                             (ts_obs, ts_aug, state_dev) float32 ...
    green                                           (ts_obs, ts_aug, state_dev) float32 ...
    blue                                            (ts_obs, ts_aug, state_dev) float32 ...
    rededge1                                        (ts_obs, ts_aug, state_dev) float32 ...
    rededge2                                        (ts_obs, ts_aug, state_dev) float32 ...
    rededge3                                        (ts_obs, ts_aug, state_dev) float32 ...
    ...                                              ...
    uvindex                                         (datetime, name) int32 ...
    severerisk                                      (datetime, name) float64 ...
    sunrise                                         (datetime, name) datetime64[ns] ...
    sunset                                          (datetime, name) datetime64[ns] ...
    moonphase                                       (datetime, name) float64 ...
    solarexposure                                   (datetime, name) int32 ...

# Create X & y

In [7]:
X, y = xdf.drop(TARGET), xdf[[TARGET]+S_COLUMNS].to_dataframe()[['Rice Yield (kg/ha)']].groupby(['ts_aug', 'ts_obs']).first()

In [8]:
y = y.reorder_levels(['ts_obs', 'ts_aug']).sort_index()

# Scale y

In [9]:
y_scaler = MinMaxScaler().fit(y)
y_scaled = y_scaler.transform(y)
y_scaled

array([[0.10714286],
       [0.10714286],
       [0.10714286],
       ...,
       [0.71428571],
       [0.71428571],
       [0.71428571]])

# Smoother

In [10]:
Smoother(mode='savgol').fit_transform(X)

<xarray.Dataset>
Dimensions:                                         (ts_obs: 557, ts_aug: 10,
                                                     state_dev: 24,
                                                     datetime: 275, name: 3)
Coordinates:
    time                                            (ts_obs, state_dev, ts_aug) object ...
  * ts_obs                                          (ts_obs) int32 0 1 ... 556
  * state_dev                                       (state_dev) int32 23 ... 0
  * ts_aug                                          (ts_aug) int32 0 1 2 ... 8 9
  * datetime                                        (datetime) object '2021-1...
  * name                                            (name) object 'Chau_Phu' ...
Data variables: (12/55)
    ndvi                                            (ts_obs, ts_aug, state_dev) float32 ...
    savi                                            (ts_obs, ts_aug, state_dev) float32 ...
    evi                                             (ts_obs, ts_aug, state_dev) float32 ...
    rep                                             (ts_obs, ts_aug, state_dev) float32 ...
    osavi                                           (ts_obs, ts_aug, state_dev) float32 ...
    rdvi                                            (ts_obs, ts_aug, state_dev) float32 ...
    ...                                              ...
    uvindex                                         (datetime, name) int32 ...
    severerisk                                      (datetime, name) float64 ...
    sunrise                                         (datetime, name) datetime64[ns] ...
    sunset                                          (datetime, name) datetime64[ns] ...
    moonphase                                       (datetime, name) float64 ...
    solarexposure                                   (datetime, name) int32 ...

# Convertor

In [11]:
df = Convertor(agg=True, weather=False).fit_transform(X)
df

Field size (ha)  Rice Crop Intensity(D=Double, T=Triple)  \
ts_obs ts_aug                                                             
0      0                   3.4                                      3.0   
       1                   3.4                                      3.0   
       2                   3.4                                      3.0   
       3                   3.4                                      3.0   
       4                   3.4                                      3.0   
...                        ...                                      ...   
556    5                   2.3                                      3.0   
       6                   2.3                                      3.0   
       7                   2.3                                      3.0   
       8                   2.3                                      3.0   
       9                   2.3                                      3.0   

               ndvi_max  ndvi_mean  ndvi_min  savi_max  savi_mean  savi_min  \
ts_obs ts_aug                                                                 
0      0       0.526096   0.339800  0.022330  1.263033   1.169885  1.011164   
       1       0.526096   0.342784 -0.011549  1.263033   1.171377  0.994226   
       2       0.576668   0.358727  0.004875  1.288311   1.179348  1.002437   
       3       0.543041   0.347012 -0.000865  1.271499   1.173490  0.999568   
       4       0.568646   0.353845  0.008548  1.284300   1.176907  1.004274   
...                 ...        ...       ...       ...        ...       ...   
556    5       0.857781   0.357780  0.014857  1.428829   1.178869  1.007428   
       6       0.661150   0.354350 -0.017497  1.330526   1.177155  0.991252   
       7       0.741412   0.372276  0.082438  1.370649   1.186116  1.041215   
       8       0.817847   0.352681 -0.021671  1.408865   1.176320  0.989166   
       9       0.864735   0.362150  0.015971  1.432307   1.181054  1.007985   

                 evi_max  evi_mean  ...  osavi_min   rdvi_max  rdvi_mean  \
ts_obs ts_aug                       ...                                    
0      0        9.624464  0.411286  ...   0.022329  48.901329  26.008966   
       1       55.381062  3.566955  ...  -0.011548  48.901329  26.280279   
       2        9.624464 -0.671296  ...   0.004874  48.901329  28.009909   
       3        9.624464  0.688180  ...  -0.000865  48.901329  26.479292   
       4        9.624464  0.744002  ...   0.008548  49.870232  27.406881   
...                  ...       ...  ...        ...        ...        ...   
556    5        4.551883  1.077509  ...   0.014857  50.557060  25.147074   
       6        4.551883  0.762690  ...  -0.017497  41.174095  24.546698   
       7       18.723934  1.736372  ...   0.082435  43.391685  26.004728   
       8        4.551883  1.065266  ...  -0.021670  48.250706  24.985300   
       9        4.551883  1.113222  ...   0.015970  51.828331  25.444818   

               rdvi_min    mtvi1_max   mtvi1_mean   mtvi1_min  lswi_max  \
ts_obs ts_aug                                                             
0      0       1.461540  7113.479492  3145.936768  211.609650  0.308699   
       1      -0.737982  7113.479492  3167.618896  -79.897675  0.308699   
       2       0.324986  7113.479492  3459.251953  -53.480156  0.308699   
       3      -0.057544  7113.479492  3195.490234  -90.451935  0.308699   
       4       0.568296  7199.322754  3327.938477  -23.989830  0.316231   
...                 ...          ...          ...         ...       ...   
556    5       1.016154  5569.200684  2842.629639   -6.352991  0.435596   
       6      -1.307646  5280.271484  2714.889893 -453.011078  0.270656   
       7       5.897933  5388.504883  2904.160889  569.082275  0.270656   
       8      -1.640732  6068.014648  2818.085938 -504.226410  0.470940   
       9       1.103214  5640.841797  2868.785156   39.617085  0.418236   

               lswi_mean  lswi_min  
ts_obs ts

# Scaler

In [12]:
Scaler(MinMaxScaler()).get_params()

{'scaler__clip': False,
 'scaler__copy': True,
 'scaler__feature_range': (0, 1),
 'scaler': MinMaxScaler()}

In [13]:
Scaler(MinMaxScaler()).set_params(scaler=StandardScaler()).get_params()

{'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'scaler': StandardScaler()}

# Pipeline

In [25]:
step_list = [
    ('smoothor', Smoother()),
    ('convertor', Convertor()),
    ('scaler', StandardScaler()),
    # ('dim_reductor', UMAP()),
    ('estimator', XGBRegressor())
]

pipe = Pipeline(step_list)

In [26]:
def get_index_test():
    path = os.path.join(DATA_PATH, 'index.json')
    with open(path, 'r') as f:
        index_dict = json.loads(f.read())
    index_0 = np.asarray(index_dict['clusters'][0]['train']['ts_obs'])
    index_1 = np.asarray(index_dict['clusters'][1]['train']['ts_obs'])
    sub_index_0 = np.random.choice(index_0, size=int(len(index_0)/ 2), replace=False)
    sub_index_1 = np.random.choice(index_1, size=int(len(index_1)/ 2), replace=False)
    
    return np.concatenate([sub_index_0, sub_index_1], axis=0)

index_test = get_index_test()

In [27]:
index_train = [idx for idx in y.index.get_level_values('ts_obs').unique() if not idx in index_test]
y_train = y.loc[index_train, :].reorder_levels(['ts_obs', 'ts_aug']).sort_index()
pipe = pipe.fit(X.sel(ts_obs=index_train), y=y_train)
y_test = y.loc[index_test, :].reorder_levels(['ts_obs', 'ts_aug']).sort_index()
pipe.score(X.sel(ts_obs=index_test), y=y_test)

In [10]:
pipe.fit(X, y_scaled)

Pipeline(steps=[('smoothor', Smoothor(mode=None)), ('convertor', Convertor()),
                ('scaler', StandardScaler()),
                ('dim_reductor',
                 UMAP(tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})),
                ('estimator',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, co...
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=0, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
                              reg_lambda=1, ...))])